In [ ]:
import cv2
import glob
import numpy as np
import scipy.io as sio
from scipy.stats import skew
from scipy.stats import kurtosis
import pywt
from skimage.feature import greycomatrix
import scipy.io as sio
import numpy as np
import seaborn as sn
import pandas as pd
from sklearn import preprocessing
from sklearn.decomposition import KernelPCA
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.utils import to_categorical
from sklearn import metricspo
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import os
from sklearn.preprocessing import MinMaxScaler
from keras.callbacks import EarlyStopping
from sklearn.decomposition import KernelPCA
from tensorflow.keras.utils import plot_model
from IPython.display import Image
import itertools
import os
from sklearn.metrics import roc_curve, auc
kl = keras.layers
import skimage
import random
seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
tf.compat.v1.set_random_seed(seed_value)
# tf.keras.utils.set_random_seed(seed_value)
# tf.random.set_seed()

In [ ]:
def im2double(img):
    """ convert image to double format """
    min_val = np.min(img.ravel())
    max_val = np.max(img.ravel())
    out = (img.astype('float') - min_val) / (max_val - min_val)
    return out
# =============================================================================
# compute_14_features
# =============================================================================
def compute_14_features(region):
    """ Compute 14 features """
    temp_array=region.reshape(-1)
    all_pixels=temp_array[temp_array!=0]
#    Area
    Area = np.sum(all_pixels)
#    mean
    density = np.mean(all_pixels)
#   Std
    std_Density = np.std(all_pixels)
#   skewness
    Skewness = skew(all_pixels)
#   kurtosis
    Kurtosis = kurtosis(all_pixels)
#   Energy
    ENERGY =np.sum(np.square(all_pixels))
#   Entropy
    value,counts = np.unique(all_pixels, return_counts=True)
    p = counts / np.sum(counts)
    p =  p[p!=0]
    ENTROPY =-np.sum( p*np.log2(p));
#   Maximum
    MAX = np.max(all_pixels)
#   Mean Absolute Deviation
    sum_deviation= np.sum(np.abs(all_pixels-np.mean(all_pixels)))
    mean_absolute_deviation = sum_deviation/len(all_pixels)
#   Median
    MEDIAN = np.median(all_pixels)
#   Minimum
    MIN = np.min(all_pixels)
#   Range
    RANGE = np.max(all_pixels)-np.min(all_pixels)
#   Root Mean Square
    RMS = np.sqrt(np.mean(np.square(all_pixels))) 
#    Uniformity
    UNIFORMITY = np.sum(np.square(p))

    features = np.array([Area, density, std_Density,
        Skewness, Kurtosis,ENERGY, ENTROPY,
        MAX, mean_absolute_deviation, MEDIAN, MIN, RANGE, RMS, UNIFORMITY])
    return features
# =============================================================================
# GLDM
# =============================================================================
def GLDM(img, distance):
    """ GLDM in four directions """
    pro1=np.zeros(img.shape,dtype=np.float32)
    pro2=np.zeros(img.shape,dtype=np.float32)
    pro3=np.zeros(img.shape,dtype=np.float32)
    pro4=np.zeros(img.shape,dtype=np.float32)
    
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):

            if((j+distance)<img.shape[1]):
                pro1[i,j]=np.abs(img[i,j]-img[i,(j+distance)])
            if((i-distance)>0)&((j+distance)<img.shape[1]):
                pro2[i,j]=np.abs(img[i,j]-img[(i-distance),(j+distance)])
            if((i+distance)<img.shape[0]):
                pro3[i,j]=np.abs(img[i,j]-img[(i+distance),j])
            if((i-distance)>0)&((j-distance)>0):
                pro4[i,j]=np.abs(img[i,j]-img[(i-distance),(j-distance)])

    n=256;
    cnt, bin_edges=np.histogram(pro1[pro1!=0], bins=np.arange(n)/(n-1), density=False)
    Out1 = cnt.cumsum()
    cnt, bin_edges=np.histogram(pro2[pro2!=0], bins=np.arange(n)/(n-1), density=False)
    Out2 = cnt.cumsum()
    cnt, bin_edges=np.histogram(pro3[pro3!=0], bins=np.arange(n)/(n-1), density=False)
    Out3 = cnt.cumsum()
    cnt, bin_edges=np.histogram(pro4[pro4!=0], bins=np.arange(n)/(n-1), density=False)
    Out4 = cnt.cumsum()
    return Out1,Out2,Out3,Out4
# =============================================================================
#   show model
# =============================================================================
def show_model(model):
    plot_model(model, show_shapes=True, show_layer_names=True, to_file='model.png')
    return Image(filename='model.png')

# =============================================================================
# plot confusion matrix
# =============================================================================
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
# =============================================================================
# resizing, normalization and adaptive histogram equalization to images
# =============================================================================
import skimage.io as io
from skimage.transform import  rescale,resize
from skimage.util import img_as_uint,img_as_ubyte
from skimage.color import rgb2gray
from skimage import exposure
import os
import numpy as np
# =============================================================================
# source and destination dirs
# =============================================================================
class_name='wheeze'# crackle, both, normal
source_dir='./data_4gr//original_images//'+class_name
destination_dir='./data_4gr//original_images_preprocessed//'+class_name

# =============================================================================
# list images list from source dir
# =============================================================================
image_list=os.listdir(source_dir)# list of images
# =============================================================================
# Normalization and adaptive histogram equalization to each single image
# =============================================================================
import imageio
from PIL import Image
for img_name in image_list:
    img=io.imread(os.path.join(source_dir,img_name), as_gray=False)
    # img = imageio.imread(img)[:,:,:3]
    # print(img.shape)
    # print('len', len(img.shape))
    if len(img.shape) ==3:
        if img.shape[-1] == 4:
            # print('anh dau', img)
            img = img[:,:,:3]
            # print('chuyen4 to 3', img.shape)
            img_gray = rgb2gray(img)
        else:
            img_gray = rgb2gray(img)
    else:
        img_gray = img
    img_resized = resize(img_gray, (512, 512))#convert image size to 512*512
    img_rescaled=(img_resized-np.min(img_resized))/(np.max(img_resized)-np.min(img_resized))#min-max normalization 
    img_enhanced=exposure.equalize_adapthist(img_rescaled)#adapt hist
    img_resized_8bit=img_as_ubyte(img_enhanced)
    io.imsave(os.path.join(destination_dir,img_name),img_resized_8bit)#save enhanced image to destination dir   

In [ ]:
# =============================================================================
# feature extraction to create feature pool
# =============================================================================
import skimage.io as io
from skimage.transform import  rescale,resize
from skimage.util import img_as_uint,img_as_ubyte
from skimage.color import rgb2gray
from skimage import exposure
from sklearn.preprocessing import MinMaxScaler
import os
import numpy as np
from utils import*

import librosa
import librosa.display

# =============================================================================
# source dir and output file name
# =============================================================================
class_name='normal'#crackle, both, normal
source_dir='./data_4gr//original_images_preprocessed//'+class_name
output_file_name=class_name
# =============================================================================
# set labels
# =============================================================================
if output_file_name=='normal':
    label=0
elif output_file_name=='crackle':
    label=1
elif output_file_name=='wheeze':
    label=2    
else:
    label=3
# =============================================================================
# start
# =============================================================================
image_list=os.listdir(source_dir)#list of images

feature_pool=np.empty([1,322])#feature pool [1,252]
for idx,img_name in enumerate(image_list):
    # print(idx)
    img=io.imread(os.path.join(source_dir,img_name))
    img_rescaled=(img-np.min(img))/(np.max(img)-np.min(img)) 
    # print(img.shape)
    texture_features=compute_14_features(img_rescaled)#texture features
    
    fft_map=np.fft.fft2(img_rescaled)
    fft_map = np.fft.fftshift(fft_map)
    fft_map = np.abs(fft_map)
    YC=int(np.floor(fft_map.shape[1]/2)+1)
    fft_map=fft_map[:,YC:int(np.floor(3*YC/2))]
    # print('fft_map: ', fft_map.shape)
    fft_features=compute_14_features(fft_map)#FFT features
    
    wavelet_coeffs = pywt.dwt2(img_rescaled,'sym4')
    cA1, (cH1, cV1, cD1) = wavelet_coeffs
    # print('cA1: ', cA1.shape)
    # print('cH1: ', cH1.shape)
    wavelet_coeffs = pywt.dwt2(cA1,'sym4')
    cA2, (cH2, cV2, cD2) = wavelet_coeffs#wavelet features
    wavelet_features=np.concatenate((compute_14_features(cA1), compute_14_features(cH1),compute_14_features(cV1),compute_14_features(cD1)
    ,compute_14_features(cA2), compute_14_features(cH2),compute_14_features(cV2),compute_14_features(cD2)), axis=0)
    
    
    gLDM1,gLDM2,gLDM3,gLDM4=GLDM(img_rescaled,10)#GLDM in four directions
    # print('gLDM1: ', gLDM1.shape)
    # hka = s
    gldm_features=np.concatenate((compute_14_features(gLDM1), compute_14_features(gLDM2),
                                  compute_14_features(gLDM3),compute_14_features(gLDM4)), axis=0)
    
    
    glcms =greycomatrix(img, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4])#GLCM in four directions
    glcm_features=np.concatenate((compute_14_features(im2double(glcms[:, :, 0, 0])), 
                                  compute_14_features(im2double(glcms[:, :, 0, 1])),
                                  compute_14_features(im2double(im2double(glcms[:, :, 0, 2]))),
                                  compute_14_features(glcms[:, :, 0, 3])), axis=0)
    
    mel_spec = librosa.feature.inverse.mel_to_stft(img_rescaled.astype(np.float))
    # Convert to decibels
    mel_spec_db = librosa.amplitude_to_db(np.abs(mel_spec), ref=np.max)

    # Extract spectral features
    spectral_centroid = librosa.feature.spectral_centroid(S=np.abs(mel_spec_db))
    spectral_bandwidth = librosa.feature.spectral_bandwidth(S=np.abs(mel_spec_db))
    spectral_contrast = librosa.feature.spectral_contrast(S=np.abs(mel_spec_db))
    spectral_rolloff = librosa.feature.spectral_rolloff(S=np.abs(mel_spec_db))
    
    spectral_features=np.concatenate((compute_14_features(spectral_centroid), compute_14_features(spectral_bandwidth),
                                  compute_14_features(spectral_contrast),compute_14_features(spectral_rolloff)), axis=0)
    # MFCC feature
    mfcc = librosa.feature.mfcc(S=librosa.power_to_db(mel_spec_db))
    mfcc_feature =  compute_14_features(mfcc) 
    
    feature_vector=np.concatenate((texture_features,fft_features,wavelet_features,gldm_features,glcm_features, spectral_features, mfcc_feature), axis=0).reshape(1,322)#merge to create a feature vector of 252
    feature_pool=np.concatenate((feature_pool,feature_vector), axis=0)

feature_pool=np.delete(feature_pool, 0, 0)
feature_pool=np.concatenate((feature_pool,label*np.ones(len(feature_pool)).reshape(len(feature_pool),1)), axis=1)#add label to the last column   
sio.savemat(output_file_name + '_322.mat', {output_file_name: feature_pool})#save the created feature pool as a mat file  

In [ ]:
# =============================================================================
# evaluate extracted features on mat files
# =============================================================================
import scipy.io as sio
import os
import seaborn as sn
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc , classification_report
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
# =============================================================================
# source dir
# =============================================================================
source_dir='./'
# =============================================================================
# load mat files
# =============================================================================
normal_features=sio.loadmat(os.path.join(source_dir,'normal_322.mat')) 
normal_features=normal_features['normal']

crackle_features=sio.loadmat(os.path.join(source_dir,'crackle_322.mat')) 
crackle_features=crackle_features['crackle']

wheeze_features=sio.loadmat(os.path.join(source_dir,'wheeze_322.mat')) 
wheeze_features=wheeze_features['wheeze']

both_features=sio.loadmat(os.path.join(source_dir,'both_322.mat')) 
both_features=both_features['both']  

# pneumonia_features=sio.loadmat(os.path.join(source_dir,'pneumonia.mat')) 
# pneumonia_features=pneumonia_features['pneumonia']    

scores=np.concatenate((normal_features[:,:-1], crackle_features[:,:-1], wheeze_features[:,:-1],both_features[:,:-1]), axis=0)
targets=np.concatenate((normal_features[:,-1],crackle_features[:,-1], wheeze_features[:,-1], both_features[:,-1]), axis=0)
# =============================================================================
# Normalization
# =============================================================================
min_max_scaler=MinMaxScaler()
scores = min_max_scaler.fit_transform(scores) 
# =============================================================================
# correlation map and histogram 
# =============================================================================
df = pd.DataFrame(scores)             
corrMatrix = df.corr().abs().fillna(0)
fig = plt.figure()
sn.heatmap(corrMatrix,xticklabels=50,yticklabels=50,cmap='jet')
# plt.savefig('corr_map',dpi=300,format='eps')
plt.savefig('cor_Map.jpg',dpi=300)
# 
fig = plt.figure()
(h,x)=np.histogram(corrMatrix.to_numpy().reshape(1,322*322), bins=8) 
# (h, x) = np.histogram(corrMatrix.to_numpy().reshape(1, -1), bins=8)

plt.bar(x[1:],h/(322*322),width=0.1)            
plt.grid(linewidth=.3)  
plt.xlabel('Correlation coefficient value')
plt.ylabel('Frequency percentage (%)')
plt.savefig('corr_hist.jpg',dpi=300)
# =============================================================================
# auc value graphs where positive label is normal             
# =============================================================================
pos_label=0   #normal
roc_list=[]  
for idx in range(scores.shape[1]):               
    fpr, tpr, thresholds = roc_curve(targets, scores[:,idx], pos_label=pos_label)
    auc_value=auc(fpr, tpr)
    if auc_value<0.5:
        auc_value=1-auc_value
    roc_list.append(auc_value)# calculate auc value

print(np.argmax(roc_list))
print(np.max(roc_list))
fig = plt.figure()
plt.bar([1,2,3,4,5,6,7],[np.mean(roc_list[:14]),np.mean(roc_list[14:28])
        ,np.mean(roc_list[28:140]),np.mean(roc_list[140:196]),np.mean(roc_list[196:252]), np.mean(roc_list[252:308]), 
                         np.mean(roc_list[308:322])] ,width=0.5)

plt.xticks([1,2,3,4,5,6,7], ('Texture', 'FFT', 'Wavelet', 'GLDM', 'GLCM', 'Spectral', 'MFCC'))
plt.grid(linewidth=.3)
plt.ylim([0, 1])
plt.title('pos_label=Normal')
plt.ylabel('Average AUC value')
plt.savefig('avg_bar_pos_normal.jpg',dpi=300)

fig = plt.figure()
plt.plot([i for i in range(1,scores.shape[1]+1)],np.sort(roc_list)[::-1],'x')
plt.grid(linewidth=.3)
plt.xlim([0.0, scores.shape[1]+1])
plt.ylim([0.5, 1])
plt.xlabel('Features')
plt.ylabel('Sorted AUC values')
plt.title('pos_label=Normal')
plt.savefig('auc_pos_normal.jpg',dpi=300)

# =============================================================================
# auc value graphs where positive label is crackle
# =============================================================================
pos_label=1   # crackle
roc_list=[]  
for idx in range(scores.shape[1]):               
    fpr, tpr, thresholds = roc_curve(targets, scores[:,idx], pos_label=pos_label)
    auc_value=auc(fpr, tpr)
    if auc_value<0.5:
        auc_value=1-auc_value
    roc_list.append(auc_value)# calculate auc value

fig = plt.figure()
plt.bar([1,2,3,4,5,6,7],[np.mean(roc_list[:14]),np.mean(roc_list[14:28])
        ,np.mean(roc_list[28:140]),np.mean(roc_list[140:196]),np.mean(roc_list[196:252]), np.mean(roc_list[252:308]), 
                         np.mean(roc_list[308:322])] ,width=0.5)

plt.xticks([1,2,3,4,5,6,7], ('Texture', 'FFT', 'Wavelet', 'GLDM', 'GLCM', 'Spectral', 'MFCC'))
plt.grid(linewidth=.3)
plt.ylim([0, 1])
plt.title('pos_label=Abnormal')
plt.ylabel('Average AUC value')
plt.savefig('avg_bar_pos_abnormal.jpg',dpi=300)

# =============================================================================
# auc value graphs where positive label is wheeze
# =============================================================================
pos_label=2   # wheeze
roc_list=[]  
for idx in range(scores.shape[1]):               
    fpr, tpr, thresholds = roc_curve(targets, scores[:,idx], pos_label=pos_label)
    auc_value=auc(fpr, tpr)
    if auc_value<0.5:
        auc_value=1-auc_value
    roc_list.append(auc_value)# calculate auc value

fig = plt.figure()
plt.bar([1,2,3,4,5,6,7],[np.mean(roc_list[:14]),np.mean(roc_list[14:28])
        ,np.mean(roc_list[28:140]),np.mean(roc_list[140:196]),np.mean(roc_list[196:252]), np.mean(roc_list[252:308]), 
                         np.mean(roc_list[308:322])] ,width=0.5)

plt.xticks([1,2,3,4,5,6,7], ('Texture', 'FFT', 'Wavelet', 'GLDM', 'GLCM', 'Spectral', 'MFCC'))
plt.grid(linewidth=.3)
plt.ylim([0, 1])
plt.title('pos_label=Abnormal')
plt.ylabel('Average AUC value')
plt.savefig('avg_bar_pos_abnormal.jpg',dpi=300)

# =============================================================================
# auc value graphs where positive label is both
# =============================================================================
pos_label = 3   # both
roc_list = []  
for idx in range(scores.shape[1]):               
    fpr, tpr, thresholds = roc_curve(targets, scores[:,idx], pos_label=pos_label)
    auc_value=auc(fpr, tpr)
    if auc_value<0.5:
        auc_value=1-auc_value
    roc_list.append(auc_value)# calculate auc value

fig = plt.figure()
plt.bar([1,2,3,4,5,6,7],[np.mean(roc_list[:14]),np.mean(roc_list[14:28])
        ,np.mean(roc_list[28:140]),np.mean(roc_list[140:196]),np.mean(roc_list[196:252]), np.mean(roc_list[252:308]), 
                         np.mean(roc_list[308:322])] ,width=0.5)

plt.xticks([1,2,3,4,5,6,7], ('Texture', 'FFT', 'Wavelet', 'GLDM', 'GLCM', 'Spectral', 'MFCC'))
plt.grid(linewidth=.3)
plt.ylim([0, 1])
plt.title('pos_label=Abnormal')
plt.ylabel('Average AUC value')
plt.savefig('avg_bar_pos_abnormal.jpg',dpi=300)

print(np.argmax(roc_list))
print(np.max(roc_list))
fig = plt.figure()
plt.plot([i for i in range(1,scores.shape[1]+1)],np.sort(roc_list)[::-1],'x')
plt.grid(linewidth=.3)
plt.xlim([0.0, scores.shape[1]+1])
plt.ylim([0.5, 1])
plt.xlabel('Features')
plt.ylabel('Sorted AUC values')
plt.title('pos_label=Abnormal')
plt.savefig('auc_pos_abnormal.jpg',dpi=300)


In [ ]:
from utils import*
# =============================================================================
# source_dir
# =============================================================================
source_dir= './'
# =============================================================================
# load mat files
# =============================================================================
normal_features=sio.loadmat(os.path.join(source_dir,'normal_322.mat')) 
normal_features=normal_features['normal']

crackle_features=sio.loadmat(os.path.join(source_dir,'crackle_322.mat')) 
crackle_features=crackle_features['crackle']

wheeze_features=sio.loadmat(os.path.join(source_dir,'wheeze_322.mat')) 
wheeze_features=wheeze_features['wheeze']

both_features=sio.loadmat(os.path.join(source_dir,'both_322.mat')) 
both_features=both_features['both']    

X = np.concatenate((normal_features[:,:-1], crackle_features[:,:-1], wheeze_features[:,:-1],both_features[:,:-1]), axis=0)
y = np.concatenate((normal_features[:,-1],crackle_features[:,-1], wheeze_features[:,-1], both_features[:,-1]), axis=0)
print(y.shape)
# =============================================================================
# normalization
# =============================================================================
min_max_scaler=MinMaxScaler()
X = min_max_scaler.fit_transform(X) 
# =============================================================================
# feature reduction (K-PCA)
# =============================================================================
transformer = KernelPCA(n_components=97, kernel='linear') #30% of 322 = 97
X = transformer.fit_transform(X)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers as kl
from sklearn.model_selection import train_test_split
tf.random.set_seed(42)
# Define function to build model with specified random seed
def build_model(feature_size, n_classes, dropout):
    """ Build a small model for multi-label classification """
    inp = kl.Input((feature_size,))
    x = kl.Dense(1024, activation='relu')(inp)
    x = kl.BatchNormalization()(x)
    x = kl.Dropout(dropout)(x)
    x = kl.Dense(640, activation='relu')(x)
    x = kl.BatchNormalization()(x)
    x = kl.Dropout(dropout)(x)
    x = kl.Dense(512, activation='relu')(x)
    x = kl.BatchNormalization()(x)
    x = kl.Dropout(dropout)(x)
    x = kl.Dense(256, activation='relu')(x)
    x = kl.BatchNormalization()(x)
    x = kl.Dropout(dropout)(x)
    x = kl.Dense(128, activation='relu')(x)
    x = kl.BatchNormalization()(x)
    x = kl.Dropout(dropout)(x)
    x = kl.Dense(64, activation='relu')(x)
    x = kl.BatchNormalization()(x)
    x = kl.Dropout(dropout)(x)
    x = kl.Dense(32, activation='relu')(x)
    x = kl.BatchNormalization()(x)
    x = kl.Dropout(dropout)(x)
    out = kl.Dense(n_classes, activation='softmax')(x) # change softmax
    model = keras.Model(inputs=inp, outputs=out)
    return model

In [ ]:
# =============================================================================
# devide data into test,train, and validation sets
# =============================================================================
y = to_categorical(y)
print('y value: ', y.shape)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.4, random_state=1)
print('X train - y train:', X_train.shape, y_train.shape)
print('X test - y test:', X_test.shape, y_test.shape)

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.25, random_state=1)# 0.25
print('X train - val', X_train.shape, y_train.shape)

In [ ]:

# =============================================================================
# build model
# =============================================================================
model = build_model(feature_size=X_train.shape[-1], n_classes=4, dropout= 0.2)
print('Built model!!!', model)
# # =============================================================================
# train model
# =============================================================================
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
opt = tf.keras.optimizers.Adam(lr=0.0001)
criterion = tf.keras.losses.categorical_crossentropy
model.compile(optimizer=opt, loss=criterion,metrics=['acc'])
trainedmodel = model.fit(X_train, y_train,batch_size = 128,epochs=100, validation_data = (X_val, y_val), callbacks=[callback])


In [ ]:
fig = plt.figure()
plt.plot(model.history.history['val_loss'], 'r',model.history.history['loss'], 'b')
plt.xlabel('Epochs')
plt.ylabel('Loss Score')
plt.grid(1)
plt.savefig('training_loss.jpg',dpi=300)


fig = plt.figure()
accuracy = trainedmodel.history['acc']
epochs = range(len(accuracy))
plt.plot(epochs, accuracy )
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.grid(1)
plt.savefig('Acc.jpg',dpi=300)
# =============================================================================
# evaluate model
# =============================================================================
print('x test:', X_test.shape)
Y_Score=model.predict(X_test)
print('Y_score:', Y_Score.shape)
y_pred = np.argmax(Y_Score, axis=1)
print('y test: ', y_test.shape)
cm=confusion_matrix(np.argmax(y_test, axis=1),y_pred)
print(cm)

fig = plt.figure()
plot_confusion_matrix(cm,classes=['normal', 'abnormal'])
plt.savefig('conf_matrix.jpg',dpi=300)

test_loss=model.evaluate(X_test,y_test,verbose=1)#evaluate model
print(test_loss)#print test loss and metrics information

# =============================================================================
# ROC curve where positive label is 0 
# =============================================================================
pos_label=0
fpr, tpr, thresholds = roc_curve(np.argmax(y_test, axis=1), Y_Score[:,pos_label], pos_label=pos_label)
roc_auc = auc(fpr, tpr)# calculate auc value
fig = plt.figure()
plt.plot(fpr, tpr, color='darkorange', label='ROC curve (area = %0.2f)\npos_label=Normal' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.grid(1)
plt.legend(loc="lower right")
plt.savefig('roc_Normal.jpg',dpi=300)

# =============================================================================
# ROC curve where positive label is normal
# =============================================================================
pos_label=1
fpr, tpr, thresholds = roc_curve(np.argmax(y_test, axis=1), Y_Score[:,pos_label], pos_label=pos_label)
roc_auc = auc(fpr, tpr)# calculate auc value
fig = plt.figure()
plt.plot(fpr, tpr, color='darkorange', label='ROC curve (area = %0.2f)\npos_label=Abnormal' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.grid(1)
plt.legend(loc="lower right")
plt.savefig('roc_abnormal.jpg',dpi=300)

In [ ]:
avg_cm = confusion_matrix(np.argmax(y_test, axis=1),y_pred)

se = (avg_cm[1][1] + avg_cm[2][2] + avg_cm[3][3])/(avg_cm[0][1] + avg_cm[1][1] + avg_cm[2][1] + avg_cm[3][1] 
                                                   + avg_cm[0][2] + avg_cm[1][2] + avg_cm[2][2] + avg_cm[3][2]
                                                  + avg_cm[0][3] + avg_cm[1][3] + avg_cm[2][3] + avg_cm[3][3])
sp = avg_cm[0][0]/(avg_cm[0][0] + avg_cm[1][0] + avg_cm[2][0] + avg_cm[3][0])
sc = (se+sp)/2
print('Specificity Sp:', sp)
print('Sensitivity Se:', se)
print('Score Sc:', sc)